In [48]:
import numpy as np
import pygame
import src.simulation as sim
import src.transformations as t
import src.quaternion as q
import src.interpolation as i
import importlib

importlib.reload(t)
importlib.reload(sim)
importlib.reload(q)
importlib.reload(i)


<module 'src.interpolation' from '/Users/alfred/Documents/rotations/src/interpolation.py'>

## Settings

In [49]:
screen_width = 800
screen_height = 800
center = np.array([300., 300., 300.])
camera_location = np.array([400., 400., -100.])
vertices, faces = sim.init_cuboid(center, 25, 25, 25)

In [50]:
orientation = np.array([1., 0., 0., 0.])
orientation_keyframes = np.full((2, 4), np.nan, dtype=np.float64)
center_keyframes = np.full((2, 3), np.nan, dtype=np.float64)
reference_vertices = vertices - center

orientation_keyframes, center_keyframes, reference_vertices, orientation

(array([[nan, nan, nan, nan],
        [nan, nan, nan, nan]]),
 array([[nan, nan, nan],
        [nan, nan, nan]]),
 array([[ 25.,  25.,  25.],
        [ 25.,  25., -25.],
        [-25.,  25.,  25.],
        [-25.,  25., -25.],
        [ 25., -25.,  25.],
        [ 25., -25., -25.],
        [-25., -25.,  25.],
        [-25., -25., -25.]]),
 array([1., 0., 0., 0.]))

## Display First Frame

In [51]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(vertices), faces, screen_height)

pygame.display.update()

## Record orientation and center of first keyframe

In [52]:
orientation_keyframes[0] = orientation
center_keyframes[0] = center

orientation_keyframes, center_keyframes, 

(array([[ 1.,  0.,  0.,  0.],
        [nan, nan, nan, nan]]),
 array([[300., 300., 300.],
        [ nan,  nan,  nan]]))

### Check quaternion of the first frame

In [53]:
first_frame_vertices = q.quaternion_to_vector(q.qvq_inverse(orientation_keyframes[0], q.vector_to_quaternion(reference_vertices))) + center_keyframes[0]

first_frame_vertices

array([[325., 325., 325.],
       [325., 325., 275.],
       [275., 325., 325.],
       [275., 325., 275.],
       [325., 275., 325.],
       [325., 275., 275.],
       [275., 275., 325.],
       [275., 275., 275.]])

In [54]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(first_frame_vertices), faces, screen_height)

pygame.display.update()

## Move object & Display

In [55]:
translation = 300
degree = - np.pi / 10
t.translation(center, vertices, translation, 0)
t.rotation_about_x(center, vertices, degree)
orientation = q.quaternion_multiplication(q.rotation_quaternion(degree, np.array([1., 0., 0.])), orientation)

vertices, center, orientation

(array([[625.        , 316.05098805, 331.50183777],
        [625.        , 331.50183777, 283.94901195],
        [575.        , 316.05098805, 331.50183777],
        [575.        , 331.50183777, 283.94901195],
        [625.        , 268.49816223, 316.05098805],
        [625.        , 283.94901195, 268.49816223],
        [575.        , 268.49816223, 316.05098805],
        [575.        , 283.94901195, 268.49816223]]),
 array([600., 300., 300.]),
 array([0.98768834, 0.15643447, 0.        , 0.        ]))

In [56]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(vertices), faces, screen_height)

pygame.display.update()

## Record orientation and center of second keyframe

In [57]:
orientation_keyframes[1] = orientation
center_keyframes[1] = center

orientation_keyframes, center_keyframes

(array([[1.        , 0.        , 0.        , 0.        ],
        [0.98768834, 0.15643447, 0.        , 0.        ]]),
 array([[300., 300., 300.],
        [600., 300., 300.]]))

In [58]:
second_frame_vertices = q.quaternion_to_vector(q.qvq_inverse(orientation_keyframes[1], q.vector_to_quaternion(reference_vertices))) + center_keyframes[1]

second_frame_vertices

array([[625.        , 316.05098805, 331.50183777],
       [625.        , 331.50183777, 283.94901195],
       [575.        , 316.05098805, 331.50183777],
       [575.        , 331.50183777, 283.94901195],
       [625.        , 268.49816223, 316.05098805],
       [625.        , 283.94901195, 268.49816223],
       [575.        , 268.49816223, 316.05098805],
       [575.        , 283.94901195, 268.49816223]])

In [59]:
second_frame_vertices == vertices

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [60]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(second_frame_vertices), faces, screen_height)

pygame.display.update()

## Linear Interpolation

In [61]:
interpolation_count = 100
interpolated_quaternions = i.linear_interpolation(orientation_keyframes, interpolation_count)
interpolated_centers = i.linear_interpolation(center_keyframes, interpolation_count)

interpolated_quaternions, interpolated_centers

(array([[1.        , 0.        , 0.        , 0.        ],
        [0.99987564, 0.00158015, 0.        , 0.        ],
        [0.99975128, 0.00316029, 0.        , 0.        ],
        [0.99962692, 0.00474044, 0.        , 0.        ],
        [0.99950256, 0.00632058, 0.        , 0.        ],
        [0.9993782 , 0.00790073, 0.        , 0.        ],
        [0.99925384, 0.00948088, 0.        , 0.        ],
        [0.99912948, 0.01106102, 0.        , 0.        ],
        [0.99900512, 0.01264117, 0.        , 0.        ],
        [0.99888076, 0.01422132, 0.        , 0.        ],
        [0.9987564 , 0.01580146, 0.        , 0.        ],
        [0.99863204, 0.01738161, 0.        , 0.        ],
        [0.99850768, 0.01896175, 0.        , 0.        ],
        [0.99838332, 0.0205419 , 0.        , 0.        ],
        [0.99825896, 0.02212205, 0.        , 0.        ],
        [0.9981346 , 0.02370219, 0.        , 0.        ],
        [0.99801024, 0.02528234, 0.        , 0.        ],
        [0.997

## Display first interpolated frame

In [62]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(q.quaternion_to_vector(q.qvq_inverse(interpolated_quaternions[0], q.vector_to_quaternion(reference_vertices))) + interpolated_centers[0]), faces, screen_height)

pygame.display.update()

## Display last interpolated frame

In [63]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

screen.fill((255, 255, 255))

sim.draw_wireframe(screen, sim.projection(0, camera_location)(q.quaternion_to_vector(q.qvq_inverse(interpolated_quaternions[-1], q.vector_to_quaternion(reference_vertices))) + interpolated_centers[-1]), faces, screen_height)

pygame.display.update()

## Play

In [64]:
fps = 20
clock = pygame.time.Clock()

In [66]:
pygame.init()

screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Quaternion Test')

for j in range(interpolation_count):
    sim.render(screen, q.quaternion_to_vector(q.qvq_inverse(interpolated_quaternions[j], q.vector_to_quaternion(reference_vertices))) + interpolated_centers[j], faces, screen_height, sim.projection(0, camera_location))
    
    clock.tick(fps)